In [1]:
import sys
sys.executable

'/Users/marcelc/Documents/10_sem/pisr/pro/pisr_venv/bin/python'

In [2]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from difflib import SequenceMatcher
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')

/Users/marcelc/Documents/10_sem/pisr/pro/pisr_venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Load data

In [3]:
dir_name = '../data'

In [4]:
file_name = 'train_full.csv'
train_full = pd.read_csv(os.path.join(dir_name, file_name))
train_full.head()

,customer_id,gender,status_x,verified_x,created_at_x,updated_at_x,location_number,location_type,latitude_x,longitude_x,...,country_id,city_id,created_at_y,updated_at_y,device_type,display_orders,location_number_obj,id_obj,CID X LOC_NUM X VENDOR,target
0,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,...,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1,0,4,TCHWPBT X 0 X 4,0
1,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,...,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1,0,13,TCHWPBT X 0 X 13,0
2,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,...,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1,0,20,TCHWPBT X 0 X 20,0
3,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,...,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1,0,23,TCHWPBT X 0 X 23,0
4,TCHWPBT,Male,1,1,2018-02-07 19:16:23,2018-02-07 19:16:23,0,Work,-96.44,-67.2,...,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1,0,28,TCHWPBT X 0 X 28,0


In [5]:
train_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5802400 entries, 0 to 5802399
Data columns (total 73 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   customer_id             object 
 1   gender                  object 
 2   status_x                int64  
 3   verified_x              int64  
 4   created_at_x            object 
 5   updated_at_x            object 
 6   location_number         int64  
 7   location_type           object 
 8   latitude_x              float64
 9   longitude_x             float64
 10  id                      int64  
 11  authentication_id       float64
 12  latitude_y              float64
 13  longitude_y             float64
 14  vendor_category_en      object 
 15  vendor_category_id      float64
 16  delivery_charge         float64
 17  serving_distance        float64
 18  is_open                 float64
 19  OpeningTime             object 
 20  OpeningTime2            object 
 21  prepration_time         int64  

In [6]:
file_name = 'orders.csv'
orders = pd.read_csv(os.path.join(dir_name, file_name))
orders.head()

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135303 entries, 0 to 135302
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   akeed_order_id                  135233 non-null  float64
 1   customer_id                     135303 non-null  object 
 2   item_count                      128378 non-null  float64
 3   grand_total                     135303 non-null  float64
 4   payment_mode                    135303 non-null  int64  
 5   promo_code                      4305 non-null    object 
 6   vendor_discount_amount          135303 non-null  float64
 7   promo_code_discount_percentage  65880 non-null   float64
 8   is_favorite                     100108 non-null  object 
 9   is_rated                        135303 non-null  object 
 10  vendor_rating                   45220 non-null   float64
 11  driver_rating                   135303 non-null  float64
 12  deliverydistance

## Filter columns and join datasets

In [11]:
# Choose some columns from train_full (train_full.csv)
# vendor_rating - > mean_rating
ds_train_full = train_full[['customer_id', 'gender', 'location_type', 'id', 'OpeningTime', 'language', 'vendor_rating', 'serving_distance', 'vendor_tag_name', 'delivery_charge']]
ds_train_full.rename(columns = {"vendor_rating": "mean_rating"}, inplace = True)

# Make derived variables 'all' with id and customer_id columns
cols = ['customer_id', 'id']
ds_train_full['all'] = ds_train_full[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

print(ds_train_full.shape)
ds_train_full.head()

(5802400, 11)


,customer_id,gender,location_type,id,OpeningTime,language,mean_rating,serving_distance,vendor_tag_name,delivery_charge,all
0,TCHWPBT,Male,Work,4,11:00AM-11:30PM,EN,4.4,6.0,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",0.0,TCHWPBT_4
1,TCHWPBT,Male,Work,13,08:30AM-10:30PM,EN,4.7,5.0,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",0.7,TCHWPBT_13
2,TCHWPBT,Male,Work,20,08:00AM-10:45PM,EN,4.5,8.0,"Breakfast,Desserts,Free Delivery,Indian",0.0,TCHWPBT_20
3,TCHWPBT,Male,Work,23,10:59AM-10:30PM,EN,4.5,5.0,"Burgers,Desserts,Fries,Salads",0.0,TCHWPBT_23
4,TCHWPBT,Male,Work,28,11:00AM-11:45PM,EN,4.4,15.0,Burgers,0.7,TCHWPBT_28


In [12]:
# Drop duplicates based 'all' derived variables
ds_train_full.drop_duplicates(['all'], inplace=True)
print(ds_train_full.shape)

(3452300, 11)


In [13]:
# Choose some columns from orders (orders.csv)
# vendor_id - > id
ds_orders = orders[['akeed_order_id', 'customer_id', 'vendor_id', 'item_count', 'grand_total', 'vendor_rating']][:]
ds_orders.rename(columns = {"vendor_id": "id"}, inplace = True)

# Make derived variables 'all' with id and customer_id columns
cols = ['customer_id', 'id']
ds_orders['all'] = ds_orders[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

print(ds_orders.shape)
ds_orders.head()

(135303, 7)


,akeed_order_id,customer_id,id,item_count,grand_total,vendor_rating,all
0,163238.0,92PEE24,105,1.0,7.6,NaN,92PEE24_105
1,163240.0,QS68UD8,294,1.0,8.7,NaN,QS68UD8_294
2,163241.0,MB7VY5F,83,2.0,14.4,NaN,MB7VY5F_83
3,163244.0,KDJ951Y,90,1.0,7.1,NaN,KDJ951Y_90
4,163245.0,BAL0RVT,83,4.0,27.2,NaN,BAL0RVT_83


In [14]:
# The number of rows is dropped duplicate rows
print(ds_train_full.shape)
print(ds_orders.shape)

(3452300, 11)
(135303, 7)


In [15]:
df1 = pd.merge(ds_train_full, ds_orders, on='all', how='inner')

print(df1.shape)
df1.head()

(132027, 17)


,customer_id_x,gender,location_type,id_x,OpeningTime,language,mean_rating,serving_distance,vendor_tag_name,delivery_charge,all,akeed_order_id,customer_id_y,id_y,item_count,grand_total,vendor_rating
0,TCHWPBT,Male,Work,113,10:59AM-10:59PM,EN,4.7,15.0,"Arabic,Desserts,Free Delivery,Indian",0.0,TCHWPBT_113,200434.0,TCHWPBT,113,NaN,7.9,5.0
1,TCHWPBT,Male,Work,237,08:30PM-11:59PM,EN,4.6,15.0,"American,Burgers,Desserts,Donuts,Fries,Pasta,S...",0.7,TCHWPBT_237,170654.0,TCHWPBT,237,1.0,5.4,NaN
2,ZGFSYCZ,Male,Home,4,11:00AM-11:30PM,EN,4.4,6.0,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",0.0,ZGFSYCZ_4,11439.0,ZGFSYCZ,4,NaN,23.6,NaN
3,ZGFSYCZ,Male,Home,28,11:00AM-11:45PM,EN,4.4,15.0,Burgers,0.7,ZGFSYCZ_28,171022.0,ZGFSYCZ,28,3.0,7.9,NaN
4,ZGFSYCZ,Male,Home,28,11:00AM-11:45PM,EN,4.4,15.0,Burgers,0.7,ZGFSYCZ_28,151462.0,ZGFSYCZ,28,2.0,7.3,NaN


In [16]:
# Change some columns name and drop the same column
df1.rename(columns={"customer_id_x": "customer_id"}, inplace=True)
df1.rename(columns={"id_x": "vendor_id"}, inplace=True)
df1.drop(['customer_id_y', 'id_y'], axis=1, inplace=True)

In [18]:
# Choose some columns from ds_train_full(cleaned data)
df2 = ds_train_full[['customer_id', 'id', 'vendor_tag_name']]
df2.rename(columns={'id': 'vendor_id'}, inplace=True)

print(df2.shape)
df2.head()

(3452300, 3)


,customer_id,vendor_id,vendor_tag_name
0,TCHWPBT,4,"Arabic,Breakfast,Burgers,Desserts,Free Deliver..."
1,TCHWPBT,13,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa..."
2,TCHWPBT,20,"Breakfast,Desserts,Free Delivery,Indian"
3,TCHWPBT,23,"Burgers,Desserts,Fries,Salads"
4,TCHWPBT,28,Burgers


### Data cleaning

In [19]:
cols = ['serving_distance', 'delivery_charge', 'item_count', 'grand_total', 'vendor_rating']

def null_check(x):
    null = np.count_nonzero(
        df1[x].isnull()
    )
    return null, null/df1.shape[0]

for i in cols:
    count, percentage = null_check(i)
    print(f'{i}\n\tcount: {count}\n\tpercentage: {percentage}\n')

serving_distance
	count: 0
	percentage: 0.0

delivery_charge
	count: 0
	percentage: 0.0

item_count
	count: 6750
	percentage: 0.05112590606466859

grand_total
	count: 0
	percentage: 0.0

vendor_rating
	count: 87951
	percentage: 0.6661591947101729



In [20]:
# Drop language columns
df1.drop(['language'], axis=1, inplace=True)

# Remove null of gender columns
df1 = df1[
    df1['gender'].notnull()
].reset_index(drop=True)

In [21]:
# gender - > one-hot encoding (int)
sex = pd.get_dummies(
    df1["gender"], 
    columns=['gender'],
    prefix="sex",
    drop_first=True
)

df1 = pd.concat([df1, sex], axis=1)

# Drop the raw 'gender' column(char)
df1.drop(['gender'], axis=1, inplace=True)

In [22]:
sex.value_counts()

sex_Male
1           97382
0            7167
dtype: int64

In [23]:
df1.rename(columns={'vendor_rating': 'rating'}, inplace=True)

In [24]:
print(df1.shape)
df1.head()

(104549, 14)


,customer_id,location_type,vendor_id,OpeningTime,mean_rating,serving_distance,vendor_tag_name,delivery_charge,all,akeed_order_id,item_count,grand_total,rating,sex_Male
0,TCHWPBT,Work,113,10:59AM-10:59PM,4.7,15.0,"Arabic,Desserts,Free Delivery,Indian",0.0,TCHWPBT_113,200434.0,NaN,7.9,5.0,1
1,TCHWPBT,Work,237,08:30PM-11:59PM,4.6,15.0,"American,Burgers,Desserts,Donuts,Fries,Pasta,S...",0.7,TCHWPBT_237,170654.0,1.0,5.4,NaN,1
2,ZGFSYCZ,Home,4,11:00AM-11:30PM,4.4,6.0,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",0.0,ZGFSYCZ_4,11439.0,NaN,23.6,NaN,1
3,ZGFSYCZ,Home,28,11:00AM-11:45PM,4.4,15.0,Burgers,0.7,ZGFSYCZ_28,171022.0,3.0,7.9,NaN,1
4,ZGFSYCZ,Home,28,11:00AM-11:45PM,4.4,15.0,Burgers,0.7,ZGFSYCZ_28,151462.0,2.0,7.3,NaN,1


In [27]:
train_contents = df1[['customer_id', 'vendor_id', 'OpeningTime', 'vendor_tag_name']]

print(train_contents.shape)
train_contents.head()

(104549, 4)


,customer_id,vendor_id,OpeningTime,vendor_tag_name
0,TCHWPBT,113,10:59AM-10:59PM,"Arabic,Desserts,Free Delivery,Indian"
1,TCHWPBT,237,08:30PM-11:59PM,"American,Burgers,Desserts,Donuts,Fries,Pasta,S..."
2,ZGFSYCZ,4,11:00AM-11:30PM,"Arabic,Breakfast,Burgers,Desserts,Free Deliver..."
3,ZGFSYCZ,28,11:00AM-11:45PM,Burgers
4,ZGFSYCZ,28,11:00AM-11:45PM,Burgers


### Preprocessing the 'OpeningTime' column

In [28]:
# Split 'Openingtime' column to 2 columns (open / close)
train_contents['OpeningTime'].fillna('-', inplace=True) 

time_split = train_contents['OpeningTime'].str.split('-')
open_time = time_split.str.get(0)
close_time = time_split.str.get(1)

train_contents['open'] = open_time
train_contents['close'] = close_time

# Fill the blank to null value
train_contents['open'].fillna('',inplace=True) 
train_contents['close'].fillna('',inplace=True)

print(f'open:\n{list(train_contents.open.unique())}')
print(f'close:\n{list(train_contents.close.unique())}')

open:
['10:59AM', '08:30PM', '11:00AM', '11.30am', '09:00AM', '11AM', '10:00AM', '08:00AM', '12:00PM', '9', '', '11:59AM', '09:59AM', '05:00PM', '09:00 AM', '01.00PM', '05:30PM', '08:30AM', '8:00AM', '11:15AM', '9:00AM', '7:58AM', '04:59PM', '00:01AM', '9am', '04:00PM', '06:15AM', '06:00PM', '11:30AM']
close:
['10:59PM', '11:59PM', '11:30PM', '11:45PM', '10:30PM', '3:50pm', '11:00PM', '11PM', '3:30PM', '07:00PM', '111:00PM', '22', '', '2:15 am', '12:30PM', '10:45PM', '11:15PM', '11:45 PM', '02:.00AM', '09:30PM', '09:45PM', '08:00pm ', '03:00PM', '04:00PM', '10:00PM', '11:59PM ', '09:01AM', '11:45', '11:01PM', '01:00AM', '10pm', '1:45PM', '10:45PM ', '12:00PM', '11.59 PM']


### Make new columns 'morning, afternoon, evening' based open and close time

In [29]:
def morning_func(x):
    if x == '':
        return None
    else:
        x1 = int(x[:2].replace(':', '').replace('a', ''))
        x2 = x[-2:]
        if (x1 >= 7 and x1 <= 10) and x2 == ('AM' or 'am'): 
            return 1
        elif x1 <= 10 and len(x) <= 2:
            return 1
        else:
            return 0

def afternoon_func(x):
    if x == '':
        return None
    else:
        x1 = int(x[:2].replace(':', '').replace('a', ''))
        x2 = x[-2:]
        if x1 <= 1 and x2 == 'PM': 
            return 1
        elif x1 == 12 and x2 == 'PM':
            return 1
        elif x2 == ('AM' or 'am'):
            return 1
        elif x1 <= 10 and len(x) <= 2:
            return 1
        else:
            return 0

def evening_func(x) :
    if x == '':
        return None
    else:
        x1 = int(x[:2].replace(':', ''))
        x2 = x[-2:]
        if (x1 >= 6 and x2 == 'PM') or x2 == ('Am' or 'am'):
            return 1
        elif x1 >= 22 and len(x) <= 2:
            return 1      
        else:
            return 0

train_contents['morning'] = train_contents['open'].apply(morning_func)
train_contents['afternoon'] = train_contents['open'].apply(afternoon_func)
train_contents['evening'] = train_contents['close'].apply(evening_func)

train_contents.head()

,customer_id,vendor_id,OpeningTime,vendor_tag_name,open,close,morning,afternoon,evening
0,TCHWPBT,113,10:59AM-10:59PM,"Arabic,Desserts,Free Delivery,Indian",10:59AM,10:59PM,1.0,1.0,1.0
1,TCHWPBT,237,08:30PM-11:59PM,"American,Burgers,Desserts,Donuts,Fries,Pasta,S...",08:30PM,11:59PM,0.0,0.0,1.0
2,ZGFSYCZ,4,11:00AM-11:30PM,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",11:00AM,11:30PM,0.0,1.0,1.0
3,ZGFSYCZ,28,11:00AM-11:45PM,Burgers,11:00AM,11:45PM,0.0,1.0,1.0
4,ZGFSYCZ,28,11:00AM-11:45PM,Burgers,11:00AM,11:45PM,0.0,1.0,1.0


### If the 'breakfast' is in the vendor_tag_name and the morning value is null(blank), fill the morning and afternoon values to 1

In [30]:
# Check the null ratio of 'vendor_tag_name' 
null = np.count_nonzero(
    train_contents['vendor_tag_name'].isnull()
)
print(f'vendor_tag_name\n\tcount: {null}\n\tpercentage: {null/train_contents.shape[0]}\n')

# Remove the null value
train_contents = train_contents[
    train_contents['vendor_tag_name'].notnull()
].reset_index(drop=True)

null = np.count_nonzero(
    train_contents['vendor_tag_name'].isnull()
)
print(f'vendor_tag_name\n\tcount: {null}\n\tpercentage: {null/train_contents.shape[0]}\n')

vendor_tag_name
	count: 1611
	percentage: 0.015409042649857962

vendor_tag_name
	count: 0
	percentage: 0.0



In [31]:
# Cleaning 'vendor_tag_name'

# Change all of the chars to lower char  
train_contents['vendor_tag_name'] = train_contents['vendor_tag_name'].apply(
    lambda x:x.lower()
)

# str -> list
train_contents['vendor_tag'] = train_contents['vendor_tag_name'].str.split(',')

train_contents['vendor_tag'].head()

0            [arabic, desserts, free delivery, indian]
1    [american, burgers, desserts, donuts, fries, p...
2    [arabic, breakfast, burgers, desserts, free de...
3                                            [burgers]
4                                            [burgers]
Name: vendor_tag, dtype: object

In [32]:
# If 'breakfast' is in the tag, then add the morning
def breakfast_in_tags(tags, x2):
    if any('breakfast' in i for i in tags) and np.isnan(x2) == True:
        return 1
    else:
        return x2

train_contents['morning_2'] = train_contents.apply(
    lambda x: breakfast_in_tags(x['vendor_tag'], x['morning']), 
    axis=1
)

# afternoon 1 / remainders are not changed
for i in range(len(train_contents['afternoon'])):
    if (np.isnan(train_contents['morning'][i]) == True) and (train_contents['morning_2'][i] == 1.0):
        train_contents['afternoon'][i] = 1
    else:
        pass

# evening 0 / remainders are not changed
for i in range(len(train_contents['evening'])):
    if (np.isnan(train_contents['morning'][i]) == True) and (train_contents['morning_2'][i] == 1.0):
        train_contents['evening'][i] = 0
    else:
        pass

In [33]:
# Check the null ratio of 'vendor_tag_name' 
null = np.count_nonzero(
    train_contents['morning_2'].isnull()
)
print(f'morning_2\n\tcount: {null}\n\tpercentage: {null/train_contents.shape[0]}\n')

# Remove the null value
train_contents = train_contents[
    train_contents['morning_2'].notnull()
].reset_index(drop=True)

null = np.count_nonzero(
    train_contents['morning_2'].isnull()
)
print(f'morning_2\n\tcount: {null}\n\tpercentage: {null/train_contents.shape[0]}\n')

train_contents.drop(['morning'], axis=1, inplace=True)
train_contents.rename(columns={'morning_2': 'morning'}, inplace=True)

morning_2
	count: 487
	percentage: 0.004731003128096524

morning_2
	count: 0
	percentage: 0.0



In [35]:
print(train_contents.shape)
train_contents.head()

(102451, 10)


,customer_id,vendor_id,OpeningTime,vendor_tag_name,open,close,afternoon,evening,vendor_tag,morning
0,TCHWPBT,113,10:59AM-10:59PM,"arabic,desserts,free delivery,indian",10:59AM,10:59PM,1.0,1.0,"[arabic, desserts, free delivery, indian]",1.0
1,TCHWPBT,237,08:30PM-11:59PM,"american,burgers,desserts,donuts,fries,pasta,s...",08:30PM,11:59PM,0.0,1.0,"[american, burgers, desserts, donuts, fries, p...",0.0
2,ZGFSYCZ,4,11:00AM-11:30PM,"arabic,breakfast,burgers,desserts,free deliver...",11:00AM,11:30PM,1.0,1.0,"[arabic, breakfast, burgers, desserts, free de...",0.0
3,ZGFSYCZ,28,11:00AM-11:45PM,burgers,11:00AM,11:45PM,1.0,1.0,[burgers],0.0
4,ZGFSYCZ,28,11:00AM-11:45PM,burgers,11:00AM,11:45PM,1.0,1.0,[burgers],0.0


## Recomendation system

In [36]:
# strip 'vendor_tag' column
train_contents['vendor_tag'] = train_contents['vendor_tag'].apply(
    lambda x: [str.lower(i.replace(' ', '')) for i in x]
)

In [37]:
# Check similar words
def print_SM_similarity(word_1, word_2):
    ratio = SequenceMatcher(None, word_1, word_2).ratio()
    print(f'Similarity of {word_1} and {word_2}: {ratio}')

print_SM_similarity('pasta', 'pastas')
print_SM_similarity('pasta', 'pastry')
print_SM_similarity('pizza', 'pizzas')
print_SM_similarity('soups', 'shuwa')
print_SM_similarity('shawarma', 'shuwa')
print_SM_similarity('thali', 'thai')
print_SM_similarity('milkshakes', 'mishkak')

Similarity of pasta and pastas: 0.9090909090909091
Similarity of pasta and pastry: 0.7272727272727273
Similarity of pizza and pizzas: 0.9090909090909091
Similarity of soups and shuwa: 0.4
Similarity of shawarma and shuwa: 0.6153846153846154
Similarity of thali and thai: 0.8888888888888888
Similarity of milkshakes and mishkak: 0.7058823529411765


In [38]:
# change the word when similarity is over than 0.8 
train_contents['vendor_tag'] = train_contents['vendor_tag'].apply(
    lambda x: [i.replace('pastas', 'pasta') for i in x]
)
train_contents['vendor_tag'] = train_contents['vendor_tag'].apply(
    lambda x: [i.replace('pizzas', 'pizza') for i in x]
)
train_contents['vendor_tag'] = train_contents['vendor_tag'].apply(
    lambda x: [i.replace('thali', 'thai') for i in x]
)

train_contents['vendor_tag_1'] = train_contents['vendor_tag'].apply(
    lambda x:' '.join(x)
)

train_contents.head()

,customer_id,vendor_id,OpeningTime,vendor_tag_name,open,close,afternoon,evening,vendor_tag,morning,vendor_tag_1
0,TCHWPBT,113,10:59AM-10:59PM,"arabic,desserts,free delivery,indian",10:59AM,10:59PM,1.0,1.0,"[arabic, desserts, freedelivery, indian]",1.0,arabic desserts freedelivery indian
1,TCHWPBT,237,08:30PM-11:59PM,"american,burgers,desserts,donuts,fries,pasta,s...",08:30PM,11:59PM,0.0,1.0,"[american, burgers, desserts, donuts, fries, p...",0.0,american burgers desserts donuts fries pasta s...
2,ZGFSYCZ,4,11:00AM-11:30PM,"arabic,breakfast,burgers,desserts,free deliver...",11:00AM,11:30PM,1.0,1.0,"[arabic, breakfast, burgers, desserts, freedel...",0.0,arabic breakfast burgers desserts freedelivery...
3,ZGFSYCZ,28,11:00AM-11:45PM,burgers,11:00AM,11:45PM,1.0,1.0,[burgers],0.0,burgers
4,ZGFSYCZ,28,11:00AM-11:45PM,burgers,11:00AM,11:45PM,1.0,1.0,[burgers],0.0,burgers


In [39]:
train_contents['vendor_id'].value_counts()

113    5792
105    4822
84     4163
79     4046
78     3688
       ... 
295     390
582     387
841     379
845     365
843     339
Name: vendor_id, Length: 95, dtype: int64

In [40]:
prepared_df = train_contents.drop_duplicates('vendor_id', keep='first', inplace=False)
prepared_df['vendor_id'] = prepared_df['vendor_id'].astype(str)

print(prepared_df.shape)
prepared_df_cp1 = prepared_df[:]

(95, 11)


### TF-IDF (vendor tags)

In [41]:
prepared_df.set_index('vendor_id', inplace=True)
prepared_df.head()

,customer_id,OpeningTime,vendor_tag_name,open,close,afternoon,evening,vendor_tag,morning,vendor_tag_1
vendor_id,,,,,,,,,,
113,TCHWPBT,10:59AM-10:59PM,"arabic,desserts,free delivery,indian",10:59AM,10:59PM,1.0,1.0,"[arabic, desserts, freedelivery, indian]",1.0,arabic desserts freedelivery indian
237,TCHWPBT,08:30PM-11:59PM,"american,burgers,desserts,donuts,fries,pasta,s...",08:30PM,11:59PM,0.0,1.0,"[american, burgers, desserts, donuts, fries, p...",0.0,american burgers desserts donuts fries pasta s...
4,ZGFSYCZ,11:00AM-11:30PM,"arabic,breakfast,burgers,desserts,free deliver...",11:00AM,11:30PM,1.0,1.0,"[arabic, breakfast, burgers, desserts, freedel...",0.0,arabic breakfast burgers desserts freedelivery...
28,ZGFSYCZ,11:00AM-11:45PM,burgers,11:00AM,11:45PM,1.0,1.0,[burgers],0.0,burgers
33,ZGFSYCZ,11:00AM-10:30PM,"desserts,mexican",11:00AM,10:30PM,1.0,1.0,"[desserts, mexican]",0.0,desserts mexican


In [42]:
vectorizer = TfidfVectorizer()
count_matrix = vectorizer.fit_transform(prepared_df['vendor_tag_1'])
print(vectorizer.vocabulary_)

{'arabic': 1, 'desserts': 14, 'freedelivery': 18, 'indian': 27, 'american': 0, 'burgers': 6, 'donuts': 16, 'fries': 20, 'pasta': 44, 'salads': 49, 'sandwiches': 50, 'breakfast': 5, 'grills': 22, 'lebanese': 34, 'shawarma': 52, 'mexican': 37, 'asian': 2, 'healthyfood': 23, 'japanese': 29, 'sushi': 58, 'hotdogs': 25, 'biryani': 4, 'soups': 55, 'freshjuices': 19, 'smoothies': 54, 'cakes': 8, 'coffee': 11, 'hotchocolate': 24, 'bagels': 3, 'kidsmeal': 32, 'pizza': 46, 'familymeal': 17, 'kebabs': 31, 'rice': 47, 'cafe': 7, 'icecreams': 26, 'italian': 28, 'sweets': 59, 'thai': 60, 'vegetarian': 61, 'milkshakes': 38, 'mandazi': 36, 'omani': 41, 'steaks': 57, 'frozenyoghurt': 21, 'mojitos': 40, 'mishkak': 39, 'organic': 42, 'manakeesh': 35, 'crepes': 13, 'pancakes': 43, 'waffles': 62, 'chinese': 9, 'dimsum': 15, 'churros': 10, 'spanishlatte': 56, 'pastry': 45, 'seafood': 51, 'rolls': 48, 'karak': 30, 'combos': 12, 'kushari': 33, 'shuwa': 53}


In [43]:
indices = pd.Series(prepared_df.index)
indices[:5]

0    113
1    237
2      4
3     28
4     33
Name: vendor_id, dtype: object

In [44]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [45]:
prepared_df = prepared_df.reset_index()

def get_recommendations_df(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    recommendations_df = prepared_df[[
        'vendor_id', 'vendor_tag_name', 'open', 'close', 'morning', 'afternoon', 'evening'
    ]]

    # cosin_similarity
    sim_scores = list(cosine_sim[idx])
    recommendations_df['sim_score'] = sim_scores
    return recommendations_df.sort_values(['sim_score'], ascending=False).head(11).reset_index()


def get_recommendations(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    # cosin_similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # most similar vendor_id (10)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [46]:
get_recommendations_df('113')

,index,vendor_id,vendor_tag_name,open,close,morning,afternoon,evening,sim_score
0,0,113,"arabic,desserts,free delivery,indian",10:59AM,10:59PM,1.0,1.0,1.0,1.000000
1,36,20,"breakfast,desserts,free delivery,indian",08:00AM,10:45PM,1.0,1.0,1.0,0.718061
2,61,149,arabic,11:15AM,10:30PM,0.0,1.0,1.0,0.578124
3,30,83,"breakfast,desserts,salads,arabic",11:00AM,11:00PM,0.0,1.0,1.0,0.501533
4,42,66,"breakfast,burgers,desserts,indian,salads",05:00PM,11:00PM,0.0,0.0,1.0,0.481316
5,8,201,"biryani,desserts,indian,soups",11:00AM,3:50pm,0.0,1.0,0.0,0.422081
6,2,4,"arabic,breakfast,burgers,desserts,free deliver...",11:00AM,11:30PM,0.0,1.0,1.0,0.412896
7,73,180,"arabic,grills,indian,pasta,salads,seafood,soup...",11:00AM,11:00PM,0.0,1.0,1.0,0.386587
8,78,145,"arabic,pizzas,sandwiches",04:00PM,11:45PM,0.0,0.0,1.0,0.367570
9,56,207,"arabic,biryani,chinese,grills,indian,rice,sala...",11:00AM,04:00PM,0.0,1.0,0.0,0.354010


### TF-IDF (vendor tags + time tags)

In [47]:
cols = ['morning', 'afternoon', 'evening']

prepared_df['time'] = ''
for col_name in cols:
    prepared_df.loc[prepared_df[col_name] == 1, 'time'] = prepared_df['time'] + ' ' + col_name

prepared_df = prepared_df[['vendor_id', 'customer_id', 'vendor_tag', 'vendor_tag_1', 'vendor_tag_name', 'time']]
prepared_df.head()

,vendor_id,customer_id,vendor_tag,vendor_tag_1,vendor_tag_name,time
0,113,TCHWPBT,"[arabic, desserts, freedelivery, indian]",arabic desserts freedelivery indian,"arabic,desserts,free delivery,indian",morning afternoon evening
1,237,TCHWPBT,"[american, burgers, desserts, donuts, fries, p...",american burgers desserts donuts fries pasta s...,"american,burgers,desserts,donuts,fries,pasta,s...",evening
2,4,ZGFSYCZ,"[arabic, breakfast, burgers, desserts, freedel...",arabic breakfast burgers desserts freedelivery...,"arabic,breakfast,burgers,desserts,free deliver...",afternoon evening
3,28,ZGFSYCZ,[burgers],burgers,burgers,afternoon evening
4,33,ZGFSYCZ,"[desserts, mexican]",desserts mexican,"desserts,mexican",afternoon evening


In [48]:
# 'time' column : str -> list
prepared_df['time'] = prepared_df['time'].str.split(' ')

def remove_blank(lists):
    return [key for key in lists if key != '']

prepared_df['time'] = prepared_df['time'].apply(
    lambda x: remove_blank(x)
)

prepared_df.head()

,vendor_id,customer_id,vendor_tag,vendor_tag_1,vendor_tag_name,time
0,113,TCHWPBT,"[arabic, desserts, freedelivery, indian]",arabic desserts freedelivery indian,"arabic,desserts,free delivery,indian","[morning, afternoon, evening]"
1,237,TCHWPBT,"[american, burgers, desserts, donuts, fries, p...",american burgers desserts donuts fries pasta s...,"american,burgers,desserts,donuts,fries,pasta,s...",[evening]
2,4,ZGFSYCZ,"[arabic, breakfast, burgers, desserts, freedel...",arabic breakfast burgers desserts freedelivery...,"arabic,breakfast,burgers,desserts,free deliver...","[afternoon, evening]"
3,28,ZGFSYCZ,[burgers],burgers,burgers,"[afternoon, evening]"
4,33,ZGFSYCZ,"[desserts, mexican]",desserts mexican,"desserts,mexican","[afternoon, evening]"


In [49]:
# combine 'vendor_tag' with 'time' (str for w2v)
prepared_df['time_1'] = prepared_df['time'].apply(
    lambda x:' '.join(x)
)

prepared_df['vendor_time_tag'] = prepared_df[['vendor_tag_1', 'time_1']].apply(
    lambda x: ' '.join(x),
    axis=1
)

prepared_df = prepared_df[['vendor_id', 'customer_id', 'vendor_time_tag', 'vendor_tag', 'vendor_tag_1', 'vendor_tag_name', 'time']]

prepared_df_cp2 = prepared_df[:]

In [50]:
vectorizer = TfidfVectorizer()
count_matrix = vectorizer.fit_transform(prepared_df['vendor_time_tag'])
print(vectorizer.vocabulary_)

{'arabic': 2, 'desserts': 15, 'freedelivery': 20, 'indian': 29, 'morning': 43, 'afternoon': 0, 'evening': 18, 'american': 1, 'burgers': 7, 'donuts': 17, 'fries': 22, 'pasta': 47, 'salads': 52, 'sandwiches': 53, 'breakfast': 6, 'grills': 24, 'lebanese': 36, 'shawarma': 55, 'mexican': 39, 'asian': 3, 'healthyfood': 25, 'japanese': 31, 'sushi': 61, 'hotdogs': 27, 'biryani': 5, 'soups': 58, 'freshjuices': 21, 'smoothies': 57, 'cakes': 9, 'coffee': 12, 'hotchocolate': 26, 'bagels': 4, 'kidsmeal': 34, 'pizza': 49, 'familymeal': 19, 'kebabs': 33, 'rice': 50, 'cafe': 8, 'icecreams': 28, 'italian': 30, 'sweets': 62, 'thai': 63, 'vegetarian': 64, 'milkshakes': 40, 'mandazi': 38, 'omani': 44, 'steaks': 60, 'frozenyoghurt': 23, 'mojitos': 42, 'mishkak': 41, 'organic': 45, 'manakeesh': 37, 'crepes': 14, 'pancakes': 46, 'waffles': 65, 'chinese': 10, 'dimsum': 16, 'churros': 11, 'spanishlatte': 59, 'pastry': 48, 'seafood': 54, 'rolls': 51, 'karak': 32, 'combos': 13, 'kushari': 35, 'shuwa': 56}


In [51]:
prepared_df.set_index('vendor_id', inplace=True)
indices = pd.Series(prepared_df.index)
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [52]:
prepared_df.iloc[0]['vendor_time_tag']

'arabic desserts freedelivery indian morning afternoon evening'

In [53]:
prepared_df = prepared_df.reset_index()

def get_recommendations_df(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    recommendations_df = prepared_df[[
        'vendor_id', 'vendor_tag_name', 'time', #'time_tag'
    ]]
    # recommendations_df.rename(columns={'time_tag': 'vendor_tag + time'}, inplace=True)

    # cosin_similarity
    sim_scores = list(cosine_sim[idx])
    recommendations_df['sim_score'] = sim_scores
    return recommendations_df.sort_values(['sim_score'], ascending=False).head(11).reset_index()


def get_recommendations(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    # cosin_similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # most similar vendor_id (10)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [54]:
get_recommendations_df('113')

,index,vendor_id,vendor_tag_name,time,sim_score
0,0,113,"arabic,desserts,free delivery,indian","[morning, afternoon, evening]",1.000000
1,36,20,"breakfast,desserts,free delivery,indian","[morning, afternoon, evening]",0.764622
2,61,149,arabic,"[afternoon, evening]",0.602083
3,30,83,"breakfast,desserts,salads,arabic","[afternoon, evening]",0.528275
4,42,66,"breakfast,burgers,desserts,indian,salads",[evening],0.479594
5,2,4,"arabic,breakfast,burgers,desserts,free deliver...","[afternoon, evening]",0.427004
6,8,201,"biryani,desserts,indian,soups",[afternoon],0.415421
7,18,676,"biryani,desserts,indian,kebabs,rice","[morning, afternoon, evening]",0.409469
8,63,679,"biryani,desserts,indian,kebabs,rice","[morning, afternoon, evening]",0.409469
9,82,681,"biryani,desserts,indian,kebabs,rice","[morning, afternoon, evening]",0.409469


## doc2vec (vendor tags)

In [71]:
prepared_df = prepared_df_cp1[:]

prepared_df.set_index('vendor_id', inplace=True)

# Embedding using word2vec
corpus = prepared_df['vendor_tag']
model = Word2Vec(vector_size=10, window=1, min_count=1, workers=4) # size=4, vector_size=100
model.build_vocab(corpus)

word_vectors = model.wv
vocabs = word_vectors.key_to_index.keys()
vocabs

dict_keys(['desserts', 'sandwiches', 'burgers', 'salads', 'american', 'breakfast', 'freedelivery', 'fries', 'freshjuices', 'pasta', 'soups', 'kidsmeal', 'mojitos', 'arabic', 'shawarma', 'grills', 'milkshakes', 'icecreams', 'indian', 'pizza', 'rice', 'smoothies', 'cafe', 'biryani', 'healthyfood', 'hotdogs', 'coffee', 'donuts', 'italian', 'lebanese', 'asian', 'steaks', 'cakes', 'hotchocolate', 'omani', 'thai', 'mexican', 'sweets', 'pancakes', 'crepes', 'kebabs', 'waffles', 'mishkak', 'rolls', 'japanese', 'shuwa', 'sushi', 'dimsum', 'combos', 'karak', 'seafood', 'pastry', 'spanishlatte', 'churros', 'chinese', 'bagels', 'manakeesh', 'organic', 'frozenyoghurt', 'mandazi', 'vegetarian', 'kushari', 'familymeal'])

In [74]:
word_vectors.most_similar('donuts')

[('coffee', 0.7669479846954346),
 ('cakes', 0.5804667472839355),
 ('kushari', 0.5380702614784241),
 ('grills', 0.5289870500564575),
 ('freshjuices', 0.5241568684577942),
 ('sushi', 0.47192177176475525),
 ('pancakes', 0.4576573967933655),
 ('vegetarian', 0.4463181495666504),
 ('omani', 0.4188254177570343),
 ('bagels', 0.3994538187980652)]

In [90]:
# Mean of word vector
def vectors(document_list):
    document_embedding_list = []

    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.key_to_index:
                count += 1
                if doc2vec is None:
                    doc2vec = model.wv[word]
                else:
                    doc2vec = doc2vec + model.wv[word]

        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    return document_embedding_list

document_embedding_list = vectors(prepared_df['vendor_tag_1'])
print('Number of document vector:', len(document_embedding_list))

Number of document vector: 95


In [92]:
cosine_sim = cosine_similarity(document_embedding_list, document_embedding_list)

print('Shape of cosine similarity matrix:', cosine_sim.shape)

Shape of cosine similarity matrix: (95, 95)


In [94]:
prepared_df = prepared_df.reset_index()

def get_recommendations_df(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    recommendations_df = prepared_df[[
        'vendor_id', 'vendor_tag_name', 'open', 'close', 'morning', 'afternoon', 'evening'
    ]]

    # cosin_similarity
    sim_scores = list(cosine_sim[idx])
    recommendations_df['sim_score'] = sim_scores
    return recommendations_df.sort_values(['sim_score'], ascending=False).head(11).reset_index()


def get_recommendations(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    # cosin_similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # most similar vendor_id (10)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [95]:
get_recommendations_df('113')

,index,vendor_id,vendor_tag_name,open,close,morning,afternoon,evening,sim_score
0,0,113,"arabic,desserts,free delivery,indian",10:59AM,10:59PM,1.0,1.0,1.0,1.000000
1,30,83,"breakfast,desserts,salads,arabic",11:00AM,11:00PM,0.0,1.0,1.0,0.720379
2,56,207,"arabic,biryani,chinese,grills,indian,rice,sala...",11:00AM,04:00PM,0.0,1.0,0.0,0.681693
3,73,180,"arabic,grills,indian,pasta,salads,seafood,soup...",11:00AM,11:00PM,0.0,1.0,1.0,0.622312
4,2,4,"arabic,breakfast,burgers,desserts,free deliver...",11:00AM,11:30PM,0.0,1.0,1.0,0.610906
5,87,197,"free delivery,mexican,salads",11:59AM,10:45PM,0.0,1.0,0.0,0.588871
6,49,288,"asian,desserts,rice,salads,soups,thai",11:15AM,03:00PM,0.0,1.0,0.0,0.531793
7,91,849,"american,breakfast,burgers,cafe,desserts,free ...",,,1.0,1.0,0.0,0.513213
8,19,845,"american,breakfast,burgers,cafe,desserts,free ...",,,1.0,1.0,0.0,0.513213
9,35,841,"american,breakfast,burgers,cafe,desserts,free ...",,,1.0,1.0,0.0,0.513213


## doc2vec (vendor tags + time tags)

In [96]:
prepared_df = prepared_df_cp2[:]

prepared_df['vendor_time_tag_list'] = prepared_df['vendor_time_tag'].str.split(' ')

# Embedding using word2vec
corpus = prepared_df['vendor_time_tag_list']
model = Word2Vec(vector_size=10, window=1, min_count=1, workers=4)
model.build_vocab(corpus)

word_vectors = model.wv
vocabs = word_vectors.key_to_index.keys()

In [98]:
word_vectors.most_similar('donuts')

[('lebanese', 0.7082822918891907),
 ('mandazi', 0.6942909955978394),
 ('american', 0.6656001806259155),
 ('morning', 0.6470872163772583),
 ('coffee', 0.6158733367919922),
 ('frozenyoghurt', 0.4895918369293213),
 ('organic', 0.486417293548584),
 ('kushari', 0.4756469428539276),
 ('waffles', 0.44109442830085754),
 ('sweets', 0.4300115704536438)]

In [101]:
# Mean of word vector
def vectors(document_list):
    document_embedding_list = []

    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.key_to_index:
                count += 1
                if doc2vec is None:
                    doc2vec = model.wv[word]
                else:
                    doc2vec = doc2vec + model.wv[word]

        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    return document_embedding_list

document_embedding_list = vectors(prepared_df['vendor_time_tag'])
print('Number of document vector:', len(document_embedding_list))

cosine_sim = cosine_similarity(document_embedding_list, document_embedding_list)
print('Shape of cosine similarity matrix:', cosine_sim.shape)

Number of document vector: 95
Shape of cosine similarity matrix: (95, 95)


In [102]:
prepared_df = prepared_df.reset_index()

def get_recommendations_df(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    recommendations_df = prepared_df[[
        'vendor_id', 'vendor_tag_name', 'time', #'time_tag'
    ]]
    # recommendations_df.rename(columns={'time_tag': 'vendor_tag + time'}, inplace=True)

    # cosin_similarity
    sim_scores = list(cosine_sim[idx])
    recommendations_df['sim_score'] = sim_scores
    return recommendations_df.sort_values(['sim_score'], ascending=False).head(11).reset_index()


def get_recommendations(vendor_id, cosine_sim=cosine_sim):
    indices = pd.Series(prepared_df.index, index = prepared_df['vendor_id']).drop_duplicates() 
    # get index from vendor_id
    idx = indices[vendor_id]

    # cosin_similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # most similar vendor_id (10)
    sim_scores = sim_scores[1:11]

    return sim_scores

In [103]:
get_recommendations_df('113')

,index,vendor_id,vendor_tag_name,time,sim_score
0,0,113,"arabic,desserts,free delivery,indian","[morning, afternoon, evening]",1.000000
1,36,20,"breakfast,desserts,free delivery,indian","[morning, afternoon, evening]",0.858929
2,57,846,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
3,21,858,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
4,35,841,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
5,19,845,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
6,20,856,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
7,86,843,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
8,91,849,"american,breakfast,burgers,cafe,desserts,free ...","[morning, afternoon]",0.851622
9,83,294,"burgers,desserts,free delivery,grills,mexican","[morning, afternoon, evening]",0.817694
